# COMP5318 - Machine Learning and Data Mining: Assignment 2 - Best Algorithm

## CONTENTS

-  [0. Set up](#0)
-  [1. Obtain data](#1)
    -  [1.1. Data info](#1.1)
    -  [1.2. Load data](#1.2)
    -  [1.3. Set train/test data](#1.3)
-  [2. Pre-process data](#2)
    -  [2.1. Standardized data](#2.1)
-  [3. Best Algorithms - CNN](#3)
-  [4. Computer details](#4)
-  [5. Easy to use](#5)

## 0. Set up <a id='0'></a>

In [ ]:
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import time
import cv2

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

## 1. Obtain data <a id='1'></a>

### 1.1. Data info <a id='1.1'></a>

There is a "Fnt" folder including 62 main folders, each folder including 1016 png files as input data (which downloaded from http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/):

    EnglishFnt.tgz (51.1 MB): characters from computer fonts with 4 variations (combinations of italic, bold and normal).

### 1.2. Load data <a id='1.2'></a>

In [ ]:
# read image and resize image
def img_resizing(i_path):
    img = cv2.imread(i_path, cv2.IMREAD_GRAYSCALE)
    data = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)
    return data

# converting the images into sets of data
data = []
label = []

for i in range(62):
    path = 'English/Fnt/Sample%03d/' % (i+1)
    for filename in os.listdir(path):
        try:
            img = img_resizing(path+filename)
            tmp = img.reshape([1, img.shape[0]*img.shape[1]])
            data.append(np.asarray(tmp, dtype = "int32"))
            label.append(i)
        except:
            error_message = path + filename
            print("failed: ", error_message)
            pass

# convert data type
data_array = np.asarray(data)
new_data = np.asarray(data_array).reshape(data_array.shape[0],-1)
print("The data shape is: ", new_data.shape)
new_label = np.asarray(label)
print("The label shape is: ", new_label.shape)

### 1.3. Set train / test data <a id='1.3'></a>

In [ ]:
from sklearn.model_selection import train_test_split
X = new_data
y = new_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

# print train and test data set shape
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

## 2. Pre-process data <a id='2'></a>

### 2.1. Standardized data <a id='2.1'></a>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()#creating an object
scaler.fit(X_train)#calculate min and max value of the training data
X_train_std = scaler.transform(X_train) #apply normalisation to the training set
X_test_std = scaler.transform(X_test) #apply normalization to the test set

## 3. Best Algorithms - CNN <a id='3'></a>

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
X_train_keras = X_train_std.reshape(X_train_std.shape[0], 28, 28)
X_test_keras = X_test_std.reshape(X_test_std.shape[0], 28, 28)

X_train_keras = np.expand_dims(X_train_keras, axis=3)
X_test_keras = np.expand_dims(X_test_keras, axis=3)

y_train_keras = keras.utils.to_categorical(y_train, 62).astype('int32')
y_test_keras = keras.utils.to_categorical(y_test, 62).astype('int32')

# print shape of data and label
print("The shape of train data: ", X_train_keras.shape)
print("The shape of test data: ", X_test_keras.shape)
print("The shape of train label: ", y_train_keras.shape)
print("The shape of test label: ", y_test_keras.shape)

The shape of train data:  (50393, 28, 28, 1)
The shape of test data:  (12599, 28, 28, 1)
The shape of train label:  (50393, 62)
The shape of test label:  (12599, 62)


In [13]:
# Clear any existing TensorFlow graph from memory and set random seeds.
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
# Model 
def cnn_model():
    # Clear any existing TensorFlow graph from memory and set random seeds.
    keras.backend.clear_session()
    np.random.seed(42)
    tf.random.set_seed(42)
    
    model = Sequential()
    model.add(layers.Conv2D(filters=64, kernel_size=(7,7),padding='same', 
                            input_shape=(28,28,1), activation='relu'))
    model.add(layers.BatchNormalization(axis=-1))
    model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(layers.Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization(axis=-1))
    model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    model.add(layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization(axis=-1))
    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization(axis=-1))
    model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(units=128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(units=62, activation='softmax'))
    return model
print(cnn_model().summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 128)       204928    
                                                                 
 batch_normalization_1 (Batc  (None, 13, 13, 128)      512       
 hNormalization)                                                 
                                                        

In [17]:
# compile and fit model
lr_adam = [0.0001, 0.001]
epoch_adam = [50, 100]

lr_sgd = [0.001, 0.01]
epoch_sgd = [50, 100]

count = 1
cnn_res = {'tech': [], 'lr':[], 'epoch':[], 'accuracy_score':[], 'precision_score':[], 'recall_score':[], 'f1_score':[]}

for i in lr_adam:
    for j in epoch_adam:
        print("Adam: %d / %d ###################################################" % (count, len(lr_adam)*len(epoch_adam)))
        count += 1
        model = cnn_model()
        model.compile(loss="categorical_crossentropy", 
                      optimizer=keras.optimizers.Adam(learning_rate=i), 
                      metrics=["accuracy"])
        model.fit(X_train_keras, y_train_keras, epochs=j,
                           validation_split=0.2) # set 20% as validation seet

        y_prob = model.predict(X_test_keras)
        y_pred = np.argmax(y_prob,axis=1)
        ground_truth = np.argmax(y_test_keras,axis=1)
        
        cnn_res['tech'].append('Adam')
        cnn_res['lr'].append(i)
        cnn_res['epoch'].append(j)
        cnn_res['accuracy_score'].append(accuracy_score(ground_truth, y_pred))
        cnn_res['precision_score'].append(precision_score(ground_truth, y_pred, average='macro'))
        cnn_res['recall_score'].append(recall_score(ground_truth, y_pred, average='macro'))
        cnn_res['f1_score'].append(f1_score(ground_truth, y_pred, average='macro'))
        print("Adam with learning rate = %f, epochs = %d, the accuracy is: %f" % (i, j, accuracy_score(ground_truth, y_pred)))

count = 1
for i in lr_sgd:
    for j in epoch_sgd:
        print(" SGD: %d / %d ###################################################" % (count, len(lr_sgd)*len(epoch_sgd)))
        count += 1
        model = cnn_model()
        decay_rate = i / j
        sgd = keras.optimizers.SGD(learning_rate=i, momentum=0.8, decay=decay_rate, nesterov=False)
        model.compile(loss="categorical_crossentropy", 
                      optimizer=sgd, 
                      metrics=["accuracy"])
        model.fit(X_train_keras, y_train_keras, epochs=j,
                           validation_split=0.2) # set 20% as validation seet
        y_prob = model.predict(X_test_keras)
        y_pred = np.argmax(y_prob,axis=1)
        ground_truth = np.argmax(y_test_keras,axis=1)

        cnn_res['tech'].append('SGD')
        cnn_res['lr'].append(i)
        cnn_res['epoch'].append(j)
        cnn_res['accuracy_score'].append(accuracy_score(ground_truth, y_pred))
        cnn_res['precision_score'].append(precision_score(ground_truth, y_pred, average='macro'))
        cnn_res['recall_score'].append(recall_score(ground_truth, y_pred, average='macro'))
        cnn_res['f1_score'].append(f1_score(ground_truth, y_pred, average='macro'))
        print("SGD with learning rate = %f, epochs = %d, the accuracy is: %f" % (i, j, accuracy_score(ground_truth, y_pred)))

Adam: 1 / 4 ###################################################
Epoch 1/50
1260/1260 [==============================] - 16s 12ms/step - loss: 3.6511 - accuracy: 0.1608 - val_loss: 1.7134 - val_accuracy: 0.6671
Epoch 2/50
1260/1260 [==============================] - 13s 11ms/step - loss: 1.9806 - accuracy: 0.4839 - val_loss: 0.9703 - val_accuracy: 0.7727
Epoch 3/50
1260/1260 [==============================] - 13s 10ms/step - loss: 1.3629 - accuracy: 0.6340 - val_loss: 0.7029 - val_accuracy: 0.8060
Epoch 4/50
1260/1260 [==============================] - 13s 10ms/step - loss: 1.0620 - accuracy: 0.7085 - val_loss: 0.5964 - val_accuracy: 0.8233
Epoch 5/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.8882 - accuracy: 0.7475 - val_loss: 0.5258 - val_accuracy: 0.8419
Epoch 6/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.7885 - accuracy: 0.7660 - val_loss: 0.4790 - val_accuracy: 0.8458
Epoch 7/50
1260/1260 [==============================] - 13s 

1260/1260 [==============================] - 13s 10ms/step - loss: 0.8908 - accuracy: 0.7461 - val_loss: 0.5297 - val_accuracy: 0.8353
Epoch 6/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.7898 - accuracy: 0.7643 - val_loss: 0.4900 - val_accuracy: 0.8442
Epoch 7/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.7137 - accuracy: 0.7828 - val_loss: 0.4568 - val_accuracy: 0.8500
Epoch 8/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.6526 - accuracy: 0.7971 - val_loss: 0.4285 - val_accuracy: 0.8598
Epoch 9/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.6016 - accuracy: 0.8065 - val_loss: 0.4180 - val_accuracy: 0.8560
Epoch 10/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.5629 - accuracy: 0.8162 - val_loss: 0.4027 - val_accuracy: 0.8644
Epoch 11/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.5255 - accuracy: 0.8276 - val_loss: 0.3807 - va

1260/1260 [==============================] - 13s 10ms/step - loss: 0.1219 - accuracy: 0.9579 - val_loss: 0.3427 - val_accuracy: 0.9045
Epoch 61/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1264 - accuracy: 0.9565 - val_loss: 0.3640 - val_accuracy: 0.9041
Epoch 62/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1230 - accuracy: 0.9577 - val_loss: 0.3466 - val_accuracy: 0.9051
Epoch 63/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1197 - accuracy: 0.9589 - val_loss: 0.3549 - val_accuracy: 0.9031
Epoch 64/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1161 - accuracy: 0.9589 - val_loss: 0.3550 - val_accuracy: 0.9016
Epoch 65/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1176 - accuracy: 0.9604 - val_loss: 0.3482 - val_accuracy: 0.9052
Epoch 66/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1172 - accuracy: 0.9600 - val_loss: 0.3569 

1260/1260 [==============================] - 13s 11ms/step - loss: 0.3338 - accuracy: 0.8796 - val_loss: 0.3205 - val_accuracy: 0.8825
Epoch 15/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3228 - accuracy: 0.8823 - val_loss: 0.3243 - val_accuracy: 0.8826
Epoch 16/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3070 - accuracy: 0.8868 - val_loss: 0.3097 - val_accuracy: 0.8913
Epoch 17/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3054 - accuracy: 0.8884 - val_loss: 0.3022 - val_accuracy: 0.8924
Epoch 18/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2853 - accuracy: 0.8950 - val_loss: 0.3003 - val_accuracy: 0.8928
Epoch 19/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2824 - accuracy: 0.8955 - val_loss: 0.3024 - val_accuracy: 0.8931
Epoch 20/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2699 - accuracy: 0.9018 - val_loss: 0.3380 - val_

Epoch 19/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2808 - accuracy: 0.8974 - val_loss: 0.2990 - val_accuracy: 0.8915
Epoch 20/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2692 - accuracy: 0.9029 - val_loss: 0.4213 - val_accuracy: 0.8802
Epoch 21/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2626 - accuracy: 0.9037 - val_loss: 0.3169 - val_accuracy: 0.8906
Epoch 22/100
1260/1260 [==============================] - 13s 11ms/step - loss: 0.2593 - accuracy: 0.9064 - val_loss: 0.2983 - val_accuracy: 0.8951
Epoch 23/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2480 - accuracy: 0.9093 - val_loss: 0.2980 - val_accuracy: 0.8946
Epoch 24/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2400 - accuracy: 0.9114 - val_loss: 0.2877 - val_accuracy: 0.8982
Epoch 25/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2415 - accuracy: 0.9115 - val_

...

Epoch 29/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.4019 - accuracy: 0.8613 - val_loss: 0.3329 - val_accuracy: 0.8807
Epoch 30/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3898 - accuracy: 0.8621 - val_loss: 0.3291 - val_accuracy: 0.8825
Epoch 31/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3810 - accuracy: 0.8632 - val_loss: 0.3276 - val_accuracy: 0.8840
Epoch 32/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3774 - accuracy: 0.8654 - val_loss: 0.3386 - val_accuracy: 0.8820
Epoch 33/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3815 - accuracy: 0.8639 - val_loss: 0.3292 - val_accuracy: 0.8836
Epoch 34/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3668 - accuracy: 0.8687 - val_loss: 0.3232 - val_accuracy: 0.8874
Epoch 35/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3642 - accuracy: 0.8691 - val_loss: 0

1260/1260 [==============================] - 13s 10ms/step - loss: 0.3598 - accuracy: 0.8721 - val_loss: 0.3196 - val_accuracy: 0.8852
Epoch 34/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3542 - accuracy: 0.8750 - val_loss: 0.3119 - val_accuracy: 0.8886
Epoch 35/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3446 - accuracy: 0.8760 - val_loss: 0.3125 - val_accuracy: 0.8877
Epoch 36/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3392 - accuracy: 0.8796 - val_loss: 0.3101 - val_accuracy: 0.8924
Epoch 37/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3348 - accuracy: 0.8789 - val_loss: 0.3076 - val_accuracy: 0.8917
Epoch 38/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3306 - accuracy: 0.8801 - val_loss: 0.3087 - val_accuracy: 0.8916
Epoch 39/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.3196 - accuracy: 0.8840 - val_loss: 0.3104 

1260/1260 [==============================] - 13s 11ms/step - loss: 0.1728 - accuracy: 0.9405 - val_loss: 0.3079 - val_accuracy: 0.9018
Epoch 89/100
1260/1260 [==============================] - 13s 11ms/step - loss: 0.1698 - accuracy: 0.9418 - val_loss: 0.3117 - val_accuracy: 0.8999
Epoch 90/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1638 - accuracy: 0.9433 - val_loss: 0.3160 - val_accuracy: 0.9007
Epoch 91/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1670 - accuracy: 0.9423 - val_loss: 0.3142 - val_accuracy: 0.9040
Epoch 92/100
1260/1260 [==============================] - 13s 11ms/step - loss: 0.1643 - accuracy: 0.9440 - val_loss: 0.3119 - val_accuracy: 0.9033
Epoch 93/100
1260/1260 [==============================] - 14s 11ms/step - loss: 0.1622 - accuracy: 0.9443 - val_loss: 0.3211 - val_accuracy: 0.8996
Epoch 94/100
1260/1260 [==============================] - 13s 11ms/step - loss: 0.1599 - accuracy: 0.9448 - val_loss: 0.3209 

Epoch 43/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2058 - accuracy: 0.9260 - val_loss: 0.2994 - val_accuracy: 0.9005
Epoch 44/50
1260/1260 [==============================] - 14s 11ms/step - loss: 0.2061 - accuracy: 0.9249 - val_loss: 0.2905 - val_accuracy: 0.8996
Epoch 45/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.2042 - accuracy: 0.9284 - val_loss: 0.2980 - val_accuracy: 0.8980
Epoch 46/50
1260/1260 [==============================] - 13s 11ms/step - loss: 0.1989 - accuracy: 0.9289 - val_loss: 0.2943 - val_accuracy: 0.9001
Epoch 47/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1938 - accuracy: 0.9299 - val_loss: 0.2979 - val_accuracy: 0.8996
Epoch 48/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1943 - accuracy: 0.9303 - val_loss: 0.2978 - val_accuracy: 0.9008
Epoch 49/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1920 - accuracy: 0.9305 - val_loss: 0

1260/1260 [==============================] - 13s 10ms/step - loss: 0.1684 - accuracy: 0.9392 - val_loss: 0.3072 - val_accuracy: 0.9014
Epoch 48/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1687 - accuracy: 0.9396 - val_loss: 0.3167 - val_accuracy: 0.8993
Epoch 49/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1624 - accuracy: 0.9419 - val_loss: 0.3050 - val_accuracy: 0.9018
Epoch 50/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1652 - accuracy: 0.9409 - val_loss: 0.3142 - val_accuracy: 0.9019
Epoch 51/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1573 - accuracy: 0.9432 - val_loss: 0.3008 - val_accuracy: 0.9047
Epoch 52/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1555 - accuracy: 0.9439 - val_loss: 0.3120 - val_accuracy: 0.9057
Epoch 53/100
1260/1260 [==============================] - 13s 10ms/step - loss: 0.1559 - accuracy: 0.9429 - val_loss: 0.3106 

In [18]:
# print result
new_cnn_res = pd.DataFrame(cnn_res)
new_cnn_res

,tech,lr,epoch,accuracy_score,precision_score,recall_score,f1_score
0,Adam,0.0001,50,0.896023,0.898168,0.895615,0.894763
1,Adam,0.0001,100,0.900230,0.900808,0.900084,0.899572
2,Adam,0.0010,50,0.906104,0.907161,0.905342,0.905588
3,Adam,0.0010,100,0.904119,0.905419,0.903803,0.903309
4,SGD,0.0010,50,0.892690,0.892944,0.892368,0.891547
5,SGD,0.0010,100,0.902849,0.903077,0.902451,0.902198
6,SGD,0.0100,50,0.901024,0.901407,0.900569,0.900407
7,SGD,0.0100,100,0.906659,0.907120,0.906179,0.906228


In [21]:
# best model
model = cnn_model()
model.compile(loss="categorical_crossentropy", 
              optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              metrics=["accuracy"])
model.fit(X_train_keras, y_train_keras, epochs=50,
                   validation_split=0.2) # set 20% as validation seet

y_prob = model.predict(X_test_keras)
y_pred = np.argmax(y_prob,axis=1)
ground_truth = np.argmax(y_test_keras,axis=1)

print("MLP - accuracy score on test set: {:.3f}".format(accuracy_score(ground_truth, y_pred)))
print("MLP - precision score on test set: {:.3f}".format(precision_score(ground_truth, y_pred, average='macro')))
print("MLP - recall score on test set: {:.3f}".format(recall_score(ground_truth, y_pred, average='macro')))
print("MLP - f1 score on test set: {:.3f}".format(f1_score(ground_truth, y_pred, average='macro')))
print("MLP - confusion matrix on test set: \n", confusion_matrix(ground_truth, y_pred))

Epoch 1/50
1260/1260 [==============================] - 14s 11ms/step - loss: 1.9023 - accuracy: 0.4972 - val_loss: 0.6358 - val_accuracy: 0.7909
Epoch 2/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.7945 - accuracy: 0.7544 - val_loss: 0.5412 - val_accuracy: 0.8101
Epoch 3/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.6625 - accuracy: 0.7823 - val_loss: 0.4719 - val_accuracy: 0.8285
Epoch 4/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.5808 - accuracy: 0.8011 - val_loss: 0.4347 - val_accuracy: 0.8392
Epoch 5/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.5416 - accuracy: 0.8148 - val_loss: 0.4065 - val_accuracy: 0.8515
Epoch 6/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.4949 - accuracy: 0.8278 - val_loss: 0.3693 - val_accuracy: 0.8633
Epoch 7/50
1260/1260 [==============================] - 13s 10ms/step - loss: 0.4691 - accuracy: 0.8344 - val_loss: 0.3660 -

In [22]:
# save model
# model.save('best_algorithm.h5')

In [23]:
# pd.DataFrame(history.history).plot(figsize=(8,5))
# plt.grid(True)
# plt.gca().set_ylim(0,1)
# plt.show()

## 4. Computer details <a id='4'></a>

- Hardware
    - OS System: Windows 10 64-bit operating system
    - CPU: Intel(R) Core(TM) i7-9700KF
    - GPU: NVIDIA GeForce RTX 2070
    - RAM: 16.0 GB
    
- Software
    - Python 3.8.3
    - notebook 6.0.3

## 5. Easy to use <a id='5'></a>

In [24]:
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import time
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
# read image and resize image
def img_resizing(i_path):
    img = cv2.imread(i_path, cv2.IMREAD_GRAYSCALE)
    data = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)
    return data

# converting the images into sets of data
data = []
label = []

data_path = 'English/Fnt/'
for i in range(62):
    path = data_path + 'Sample%03d/' % (i+1)
    for filename in os.listdir(path):
        img = img_resizing(path+filename)
        tmp = img.reshape([1, img.shape[0]*img.shape[1]])
        data.append(np.asarray(tmp, dtype = "int32"))
        label.append(i)

print("############### Data Loaded ###############")

# convert data type
data_array = np.asarray(data)
new_data = np.asarray(data_array).reshape(data_array.shape[0],-1)
new_label = np.asarray(label)

# set train/test data
X = new_data
y = new_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

# standardize data
scaler = StandardScaler()#creating an object
scaler.fit(X_train)#calculate min and max value of the training data
X_train_std = scaler.transform(X_train) #apply normalisation to the training set
X_test_std = scaler.transform(X_test) #apply normalization to the test set


X_train_keras = X_train_std.reshape(X_train_std.shape[0], 28, 28)
X_test_keras = X_test_std.reshape(X_test_std.shape[0], 28, 28)

X_train_keras = np.expand_dims(X_train_keras, axis=3)
X_test_keras = np.expand_dims(X_test_keras, axis=3)

y_train_keras = keras.utils.to_categorical(y_train, 62).astype('int32')
y_test_keras = keras.utils.to_categorical(y_test, 62).astype('int32')

# load model
print("############### Model Path Input ###############")

model_path = 'best_algorithm.h5'
model = load_model(model_path)
print("############### model loaded ###############")

# evaluate model
print("############### Model Evaluation ###############")
# Check pretrained model performance on test set
y_prob = model.predict(X_test_keras)
y_pred = np.argmax(y_prob,axis=1)
ground_truth = np.argmax(y_test_keras,axis=1)

print("CNN - accuracy score on test set: {:.3f}".format(accuracy_score(ground_truth, y_pred)))
print("CNN - precision score on test set: {:.3f}".format(precision_score(ground_truth, y_pred, average='macro')))
print("CNN - recall score on test set: {:.3f}".format(recall_score(ground_truth, y_pred, average='macro')))
print("CNN - f1 score on test set: {:.3f}".format(f1_score(ground_truth, y_pred, average='macro')))
print("CNN - confusion matrix on test set: \n", confusion_matrix(ground_truth, y_pred))

############### Data Path Input ###############
Default path is: 'English/Fnt/'
*** DO NOT ENTER anything if using the above path ***
Windows Path Example: C:/Users/xxx/Downloads/5318/English/Fnt/
MacOS Path Example: /Users/xxx/Downloads/5318/English/Fnt/
Please input the data path (before Samplexxx folders) end with '/'
############### Data Loaded ###############
############### Model Path Input ###############
############### model loaded ###############
############### Model Evaluation ###############
CNN - accuracy score on test set: 0.904
CNN - precision score on test set: 0.905
CNN - recall score on test set: 0.903
CNN - f1 score on test set: 0.903
CNN - confusion matrix on test set: 
 [[164   0   0 ...   0   0   0]
 [  0 173   0 ...   0   0   0]
 [  0   0 211 ...   0   0   0]
 ...
 [  0   0   0 ... 162   0   0]
 [  0   0   0 ...   0 191   0]
 [  0   0   0 ...   0   0 142]]
